# RecordLinkage Linking - 247 & Rivals

> Leveraging the RL library to determine approximate matching over a range of fields using various string methods methods.  This specifically focuses on 247 & Rivals first.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx

#not currently using jellyfish
import jellyfish as jf

### I don't like having all of this here - should push to functions

In [2]:
outputDir = '..//scrapedData//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

## Load the id config
idConfig = json.loads(open('..//config//idConfigLink.json', "r").read())

## Load and Merge Source Files Then Create a List of Dicts for each Dataset
> This was originally set up for all of the keys in the sourcefiles.json config.  Since this file is currently only going to serve 247 & Rivals, I've hardcoded the keys to fetch

In [3]:
dataset_keys = ['sports247', 'rivals']
dfs = []
for key in dataset_keys:
    vars()[key] = fx.mergeSourceFiles (key, outputDir, sourceFiles)    
    dfs.append(vars()[key])

## Create New IDs
> This isn't elegant and I'd love to basically preprocess these collections prior to this notebook

In [4]:
fx.createNewID(idConfig['sports247'], sports247, field_agg)
fx.createNewID(idConfig['rivals'], rivals, field_agg)

## Create Dataframes
> I might move these to their own pickles so I don't constantly have to recreate these each time

In [5]:
df_sports247 = pandas.DataFrame(sports247).set_index('ID')
df_rivals = pandas.DataFrame(rivals).set_index('ID')

## Rename the indexes to something readable and predictable
> Otherwise these become 'index' and 'level_0', etc when you start to merge these.

In [12]:
df_sports247.index.name = '247_ID'
df_sports247['ID'] = df_sports247.index
df_rivals.index.name = 'rivals_ID'
df_rivals['ID'] = df_rivals.index

## Create Blockers
> I swear this isn't working.  And honestly I guess I'm ok with it not working since I'm doing string operations later on this script.  This usually takes 20+ minutes - so it is doing something...

In [13]:
indexer = recordlinkage.BlockIndex(on=['school', 'year'])
candidate_links = indexer.index(df_sports247, df_rivals)
candidate_links

MultiIndex([('paulthurston_nebraska_2012', 'tommyarmstrong_nebraska_2012'),
            ('paulthurston_nebraska_2012',    'alonzomoore_nebraska_2012'),
            ('paulthurston_nebraska_2012', 'leroyalexander_nebraska_2012'),
            ('paulthurston_nebraska_2012',   'jaredafalava_nebraska_2012'),
            ('paulthurston_nebraska_2012', 'mohammedseisay_nebraska_2012'),
            ('paulthurston_nebraska_2012',      'samcotton_nebraska_2012'),
            ('paulthurston_nebraska_2012',     'imanicross_nebraska_2012'),
            ('paulthurston_nebraska_2012',      'averymoss_nebraska_2012'),
            ('paulthurston_nebraska_2012',   'paulthurston_nebraska_2012'),
            ('paulthurston_nebraska_2012',    'thomasbrown_nebraska_2012'),
            ...
            (    'jalendaniels_troy_2013',        'jeviusterry_troy_2013'),
            (    'jalendaniels_troy_2013',      'tayloredwards_troy_2013'),
            (    'jalendaniels_troy_2013',        'jordanchunn_troy_2013

## Define Features

> These are all pretty straightforward.  The toughest one to assess is position - since the services don't always categorize players in the same way or have the same abbreviation for a single position.  Since this is often only a 2 or 3 letter string, I decided to do an exact match.

In [14]:
c = recordlinkage.Compare()

c.exact('ID', 'ID')
c.string('playerName', 'playerName', method='damerau_levenshtein', label='playerName')
c.string('school', 'school', label='college')
c.string('city', 'city', label='city')
c.exact('state', 'state', label='state')
c.string('highSchool', 'highSchool', label='highSchool')
c.exact('position', 'position', label='position')
c.exact('year', 'year', label='year')

<Compare>

In [15]:
df_sports247

,school,year,playerName,highSchool,city,state,position,height,weight,compRating,...,nationalRank,positionRank,stateRank,247Rating,247Stars,247positionRank,247stateRank,displayName,247nationalRank,ID
247_ID,,,,,,,,,,,,,,,,,,,,,
paulthurston_nebraska_2012,nebraska,2012,paulthurston,Arvada West,Arvada,CO,OT,77,275,0.9407,...,152,16,2,92,4.0,26,2,Paul Thurston,NaN,paulthurston_nebraska_2012
gregmcmullen_nebraska_2012,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,SDE,77,255,0.9099,...,269,14,18,95,4.0,6,8,Greg McMullen,139,gregmcmullen_nebraska_2012
jordanwesterkamp_nebraska_2012,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,190,0.9134,...,254,36,6,90,4.0,46,7,Jordan Westerkamp,NaN,jordanwesterkamp_nebraska_2012
imanicross_nebraska_2012,nebraska,2012,imanicross,North Hall,Gainesville,GA,RB,73,222,0.9093,...,273,15,23,87,3.0,27,40,Imani Cross,NaN,imanicross_nebraska_2012
tommyarmstrong_nebraska_2012,nebraska,2012,tommyarmstrong,Steele,Cibolo,TX,DUAL,73,200,0.9060,...,286,6,42,86,3.0,15,110,Tommy Armstrong,NaN,tommyarmstrong_nebraska_2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
brettmedforth_troy_2013,troy,2013,brettmedforth,Sylacauga Sch,Sylacauga,AL,OG,75,245,0.7333,...,NA,210,121,NaN,NaN,NaN,NaN,Brett Medforth,NaN,brettmedforth_troy_2013
cardelllue_troy_2013,troy,2013,cardelllue,Hebron,Carrollton,TX,CB,72,185,0.7333,...,NA,244,454,NaN,NaN,NaN,NaN,Cardell Lue,NaN,cardelllue_troy_2013
zachmoore_troy_2013,troy,2013,zachmoore,Coffee,Douglas,GA,ILB,73,200,0.7333,...,NA,166,262,NaN,NaN,NaN,NaN,Zach Moore,NaN,zachmoore_troy_2013


## Generate Comparison Vector Set and Output to Pickle
> This will take 60+ minutes to complete.  The output will be an 8+ GB pickle file

In [16]:
features = c.compute(candidate_links, df_sports247, df_rivals)
features.to_pickle("features.pkl")

In [ ]:
features = pandas.read_pickle("features.pkl")

In [ ]:
features.head()

In [ ]:
features['sum'] = (features['playerName'] + features['college'] + features['city'] + features['state'] + features['highSchool'] + features['year'] + features['position'])/7

In [ ]:
features.head()

In [ ]:
deaunte = features.loc[16432]

In [ ]:
deaunte.loc[13082]

In [ ]:
deaunte.loc[11073]

In [ ]:
import cleantext as clean




In [ ]:
string = 'T.C. Robinson'

clean.clean(string, no_punct=True).replace(" ", "")